# Singular Value Decomposition (CF)

CF в любом из вариантов (**user-based** / **item-based**) имеет проблемы, если матрица сильно разрежена, а также имеет проблему масштабируемости. Это не позволяет использовать решения на основе CF на очень больших данных. 

Описание проблемы разреженности  [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf), где предлагается *Singular Value Decomposition (SVD)* для решения.

---

### Как?

SVD разкладывает матрицу размера $m\times n$ в матрицы $P$, $\Sigma$ и $Q$:

\begin{equation}
R = P\Sigma Q^{\top}.
\end{equation}

$P$ и $Q$ это ортогональные матрицы и $\Sigma$ диагональная матрица состоящая из сингулярных значений рейтингов в качестве диагональных значений ([Billsus and Pazzani, 1998](https://www.ics.uci.edu/~pazzani/Publications/MLC98.pdf), [Sarwar et al. (2000)](http://files.grouplens.org/papers/webKDD00.pdf)).

![](img/svd.png)

Матрица рейтингов рассчитывается: 

\begin{equation}
R_k = P_k\Sigma_k Q_k^{\top}.
\end{equation}


---

### SVD 

> 1. Нормализованная матрица $R_{norm}$ раскладывается на $P$, $\Sigma$ и $Q$
> 2. Уменьшаем $\Sigma$ до размерности $k$ и трансформируем в $\Sigma_k$
> 3. Считаем квадратный корень из $\Sigma_k$ для получения $\Sigma_k^{\frac{1}{2}}$
> 4. Считаем финальную матрицу $P_k\Sigma_k^{\frac{1}{2}}$ и $\Sigma_k^{\frac{1}{2}}Q_k^{\top}$, которая будет использоваться для расчета рекомендаций.

---


In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile

import os
import sys

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
ratings = pd.read_csv('./data/ratings.csv')
movies = pd.read_csv('./data/movies.csv')

In [4]:
pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
669,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
def rating_matrix(ratings):
    """
    1. Запоним NaN средним рейтингом
    2. Нормализуем рейтинг относительно среднего значения
    
    :param ratings : DataFrame
    :return
        - R : Numpy array рейтингов
        - df : DataFrame рейтингов
    """
    
    # средний рейтинг
    umean = ratings.groupby(by='userId')['rating'].mean()

    # заполняем пустоты
    df = pd.crosstab(ratings.userId, ratings.movieId, ratings.rating, aggfunc=sum)
    # заполним пустоты средним рейтингом по фильму
    df = df.fillna(df.mean(axis=0))

    # нормализация по среднему значению (из рейтинга фильма вычтем средний рейтинг пользователя)
    df = df.subtract(umean, axis=0)
    
    # в numpy
    R = df.to_numpy()
    
    return R, df

# записываем результат
R, df = rating_matrix(ratings)

$R$ готовая матрица для применения

In [ ]:
df

### SVD


1. ```fit()``` - расчет SVD рейтингов и сохранение матриц P, S, Q
2. ```predict()``` - матрицы P, S и Qh для создания предикта по пользователю - элементу. Учитывая, что мы сделали вычитание фактического рейтинга из среднего по пользователю, нам необходимо будет вернуть значение, прибавив предикт к среднему рейтингу
3. ```recommend()``` - функция рекомендаций

In [66]:
class SVD:
    
    def __init__(self, umeam):
        """
        :param
            - umean : среднее значение рейтингов по пользователю
        """
        self.umean = umean
        
        # init svd 
        self.P = np.array([])
        self.S = np.array([])
        self.Qh = np.array([])
        
        # init пользователь и элемент
        self.u_factors = np.array([])
        self.i_factors = np.array([])
    
    def fit(self, R):
        """
        Fit  SVD
        """
        P, s, Qh = np.linalg.svd(R, full_matrices=False)
        
        self.P = P
        self.S = np.diag(s)
        self.Qh = Qh
        
        # скрытые факторы по пользователю (u_factors) и по элементу (i_factors)
        self.u_factors = np.dot(self.P, np.sqrt(self.S))
        self.i_factors = np.dot(np.sqrt(self.S), self.Qh)
    
    def predict(self, userid, itemid):
        """
        Предикт по пользователю
        
        :param
            - userid : пользователь
            - itemid : элемент
            
        :return
            - r_hat : predicted rating
        """
        
        # предикт вычисляется по факторам пользователя и элемента
        r_hat = np.dot(self.u_factors[userid,:], self.i_factors[:,itemid])
        
        # суммируем со средним значением 
        r_hat += self.umean[userid]
        
        return r_hat
        
    
    def recommend(self, userid):
        """
        :param
            - userid : id пользователя
        """
        
        # предикт для пользователя по факторам   
        # Поскольку SVD производился по нормализованной таблице, то мы должны добавить среднее по пользователю
        predictions = np.dot(self.u_factors[userid,:], self.i_factors) + self.umean[userid]
        
        # сортировка результата
        top_idx = np.flip(np.argsort(predictions))
        preds = predictions[top_idx]
        
        return top_idx, preds
        

Создадим SVD модель

Передадим средний рейтинг, как базовый элемент

In [67]:
umean = ratings.groupby(by='userId')['rating'].mean()

# svd
svd = SVD(umean)

# fit
svd.fit(R)


### Предикт рейтинга

In [8]:
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [79]:
# для кого делаем предикт
userid =34

# какие элементы подбираем
items = [780]

# формирование предикта
for itemid in items:
    r = svd.predict(userid=userid, itemid=itemid)
    print('предикт для пользователя ={} по элементу ={} : {}'.format(userid, itemid, r))

предикт для пользователя =34 по элементу =780 : 4.5358288770053665


### Создание рекомендаций

In [81]:
userid = 34

# сортировка предикта
top_indx, preds = svd.recommend(userid=userid)

rec_movies = movies[movies['movieId'].isin(top_indx)]

# списко элементов, которые пользователь отметил рейтингом
uitems = ratings.loc[ratings.userId == userid].movieId.to_list()

# убираем фильмы уже оцененные пользователем
top10 = np.setdiff1d(top_indx, uitems, assume_unique=True)[:10]

# топ - N
top10_idx = list(np.where(top_indx == idx)[0][0] for idx in top10)
top10_predictions = preds[top10_idx]

# добавляем название и жанр
zipped_top10 = list(zip(top10,top10_predictions))
top10 = pd.DataFrame(zipped_top10, columns=['movieId','predictions'])
List = pd.merge(top10, movies, on='movieId', how='inner')

List

,movieId,predictions,title,genres
0,2469,6.535829,Peggy Sue Got Married (1986),Comedy|Drama
1,269,6.535829,My Crazy Life (Mi vida loca) (1993),Drama
2,501,6.535829,Naked (1993),Drama
3,594,6.535829,Snow White and the Seven Dwarfs (1937),Animation|Children|Drama|Fantasy|Musical
4,2998,6.535829,Dreaming of Joseph Lees (1999),Drama|Romance
5,2441,6.535829,"Hi-Lo Country, The (1998)",Drama|Romance|Western
6,3075,6.535829,Repulsion (1965),Drama|Horror
